# Kubeflow Lightweight Components

We will use this notebook to explore the concept of lightweight components that are used in building Kubeflow pipelines. This will allow us to explore Kubeflow constructs and deployment mechanisms without having to change anything about out existing code bases. Once we finish with this exploratory example, we can then port some of our other efforts over to Kubeflow pipelines.

First we will need to ensure that the SDK is installed on the jupyter instance that we've deployed in our production Kubeflow cluster.

We will be borrowing and extending the example [Lightweight Components](https://github.com/kubeflow/pipelines/blob/master/samples/core/lightweight_component/Lightweight%20Python%20components%20-%20basics.ipynb) example contained within the [Kubeflow Pipelines repo](https://github.com/kubeflow/pipelines).

In [1]:
!pip3 install kfp --upgrade

    100% |████████████████████████████████| 81kB 4.2MB/s ta 0:00:01
    100% |████████████████████████████████| 61kB 9.0MB/s eta 0:00:01
    100% |████████████████████████████████| 51kB 5.8MB/s eta 0:00:01
  Stored in directory: /home/jovyan/.cache/pip/wheels/0c/de/42/ff15262d1fffea9d03900e4ea2641eda6ae38f54258bab54dd
  Stored in directory: /home/jovyan/.cache/pip/wheels/07/13/f3/31e9e1a25e10b8c3d04df74a01f4dbcf16a4119272cd41ba7a
  Stored in directory: /home/jovyan/.cache/pip/wheels/bd/5b/6b/20cdc06ddb10caa3a86f5804eb9a90122ae8de0bcf19a468d8
  Stored in directory: /home/jovyan/.cache/pip/wheels/2b/67/89/414471314a2d15de625d184d8be6d38a03ae1e983dbda91e84
Successfully built kfp kfp-server-api argo-models tabulate
  Found existing installation: kfp 0.1
    Uninstalling kfp-0.1:
      Successfully uninstalled kfp-0.1
You are using pip version 19.0.1, however version 19.2.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import kfp.components as comp

As pointed out in the example above there are some scoping limitations that we will need to pay attention to because we are working to use this Jupyter notebook as an example of creating Kubeflow a [Kubeflow container op](https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.dsl.html#kfp.dsl.ContainerOp)

Below we're using a trivial example of a method that sums up two `float`s. ***It is important to note that if a function operates on numbers, like the one below, the parameters _need_ to have type hints, like we've shown below.*** These supported numbers types are `[float, int, bool]`. Everything else needs to be passed as a string.

This is a nice example of the use cases for type hints in python.

In [2]:
def add(a: float, b: float) -> float:
    '''
    Add two floats and return a float
    '''
    return a + b

In [3]:
add_op = comp.func_to_container_op(add)

In [6]:
from typing import NamedTuple

def my_divmod(dividend: float, divisor: float, output_dir:str = './') -> NamedTuple('MyDivModOutput', [('quotient', float), ('remainder', float)]):
    """
    simple method to divide numbers and return the quotient
    """
    
    import numpy as np
    
    def divmod_helper(dividend, divisor):
        return np.divmod(dividend, divisor)
    
    (quotient, remainder) = divmod_helper(dividend, divisor)
    
    from tensorflow.python.lib.io import file_io
    import json
    
    metadata = {
        'outputs' : [{
            'type': 'tensorboard',
            'source': 'gs://ml-pipeline-dataset/tensorboard-train',
            }]
        }
    
    with open(output_dir + "mlpipeline-ui-metadata.json", 'w') as f:
        json.dump(metadata, f)
    
    metrics = {
        'metrics': [{
            'name': 'quotient',
            'numberValue': float(quotient),
        },{
            'name': 'remainder',
            'numberValue': float(remainder),
        }]}
    
    with file_io.FileIO(output_dir + 'mlpipeline-metrics.json', 'w') as f:
        json.dump(metrics, f)
    
    from collections import namedtuple
    divmod_output = namedtuple('MyDivModOutput', ['quotient', 'remainder'])
    return divmod_output(quotient, remainder)

In [7]:
my_divmod(100, 7)

MyDivModOutput(quotient=14, remainder=2)

Now lets do a quick conversion to a pipeline operator

In [8]:
divmod_op = comp.func_to_container_op(my_divmod, base_image='tensorflow/tensorflow:1.13.1-py3')

In [9]:
import kfp.dsl as dsl
@dsl.pipeline(
    name='Calculation pipeline',
    description='Toy example for showing the use of Kubeflow Pipelines'
)

def calc_pipeline(a='a',b='7',c='17'):
    
    add_task = add_op(a, 4)

    divmod_task = divmod_op(add_task.output, b, '/')
    
    result_task = add_op(divmod_task.outputs['quotient'], c)

In [10]:
pipeline_func = calc_pipeline
pipeline_filename = pipeline_func.__name__ = ".pipeline.zip"
import kfp.compiler as compiler
compiler.Compiler().compile(pipeline_func, pipeline_filename)

In [11]:
arguments = {'a': '7', 'b': '8'}

EXPERIMENT_NAME = 'TEST_PIPELINE'
import kfp
client = kfp.Client()
experiment = client.create_experiment(EXPERIMENT_NAME)

run_name = pipeline_func.__name__ + ' run'
run_result = client.run_pipeline(experiment.id, run_name, pipeline_filename, arguments)
